# Fine Tuning Mistral 7B Google Colab

**Autor:** Guillermo Gallego Reina

**Fecha:** 14 de Mayo de 2024

**Introducción**

---

El ajuste de LLM es cada vez más usado por las diferentes compañias con el fin de acercar los modelos a los casos de uso propios. Con la reciente aparición de métodos de ajuste como QLoRA se puede realizar estas tareas sonbre una solo GPU permitiendo realizarlas sin la necesidad de una infraestrutura compleja distribuida en múltiples GPU. 

Este notebook explicará las diferentes etapas necesarias para realizar el ajuste del modelo LLM Mistral utilizando datos de HuggingFace de Recetas de cocina colombiana en Español. 
**#LoRA**, **#GenAI**, **#LLM**, **#Mistral**, **#GPU**, **#QLoRA**, **#GoogleColab**.

---

**Tareas:**

1. **[Importar Librerias](#1)**
   - Librerias necesarias para Google Colab.

2. **[Carga de Datos](#2)**
   - 2.1 [Realizamos la configuracion necesaria para poder acceder a Hugging Face.](#3)
   - 2.2 [Cargamos los datos.](#4)
   - 2.3 [Preprocesado de los datos de acuerdo a la plantilla de Mistral](#5)

3. **[Carga el Modelo](#6)**
   - Comenzaremos cargando el modelo base.
   
4. **[Cuantización del Modelo](#7)**
   - Cuantizaremos el modelo usando el paquete BitsAndBytes de Hugging Face.
   
5. **[QLoRA](#8)**
   - Usaremos QLoRA para ajustar el adaptador LoRA sobre el modelo cuantizado.
   
6. **[Entrenamiento](#9)**
   - Gracias al uso del modelo de 4 bits, utilizaremos la clase SFTTrainer de Hugging Face para el entrenamiento del Transfomers.
   
7. **[Despliegue](#10)**
   - Una vez realizado el ajuste se subira el modelo a Hugging Face.
   
**Entorno de Trabajo:**
   - Se utilizará **Google Colab** por su fácil acceso a GPU y TPU de alto rendimiento, lo que permite realizar tareas de ciencia de datos de manera eficiente y entrenar LLM.

---


<a id="1"></a> 
# 1. Importar Librerias

___

Dado que vamos hacer uso de **Google Colab** es necesario importar librerías especifícas para llevar a cabo el Fine-Tunning.

___

In [1]:
!pip3 install datasets
!pip install bitsandbytes transformers peft accelerate
!pip install datasets trl ninja packaging
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate
from datasets import load_dataset, Dataset, DatasetDict
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 11.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 6.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple/


2024-05-08 06:53:10.743178: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 06:53:10.743298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 06:53:10.879031: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<a id="2"></a> 
# 2. Carga de Datos

En nuestro caso de uso, utilizaremos los datos de Hugging Face [somosnlp/recetas-cocina](https://huggingface.co/datasets/somosnlp/recetas-cocina?_sm_vck=01j0t80JTt8J0q4SnT4Z4Wss3FtDWP2qjF3TQMr183Nt8Z2TWtJM). Este dataset es de tipo **#Table Question Answering** y esta enfocado en la cocina Colombiana. Contiene varias columnas pero para nuestro caso nos quedaremos unicamente con las siguientes:

* **title**: Indica el plato que se va a realizar.
* **steps**: Indica los pasos en formato lista que hay que realizar para cocinar el plato anteriormente indicado.

**NOTA**: **Dado que el obtejivo de este notebook es comprender las diferentes etapas de un ajuste vamos a realizar un sample de los datos para obtener unicamente 5K registros.**

<a id="3"></a> 
## 2.1 Configuracion

___

Dado que el entorno sobre el que estamos trabajando es **GoogleColab** es necesario tener acceso en Hugging Face. Para ello deberemos:

* **Crear una cuenta en Hugging Face**
* **Crear un Token en Hugging Face con permisos de **Escritura****
* **Crear sobre GoogleColab la variable HF_TOKEN con el token creado**

En algunas ocasiones no es suficiente crear el **HF_TOKEN**, para prevenir el error se indica un comando para poder logearse directamente con el Token creado.

___

In [3]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

<a id="4"></a> 
## 2.2 Carga de Datos

___

Como hemos comentado anteriormente realizaremos un sampleo los dados debido a las limitaciones del entorno, esto serán cargados haciendo uso de la funcion **load_dataset** previamente importada.

___

In [4]:
dataset = load_dataset("somosnlp/recetas-cocina")
df = dataset['train'].to_pandas()
#Sampleamos 5K
df = df.sample(frac=0.10,random_state=1234).reset_index(drop=True)
#Renombramos las columnas de acuerdo a los datos
df = df[['title', 'steps']]
df.columns = ['context', 'output']

Generating train split: 0 examples [00:00, ? examples/s]

<a id="5"></a> 
## 2.3 Procesado

___

Como todo modelo de LLM es necesario conocer la plantilla con el cual fueron entrenados, en nuestro caso dado que seleccionaremos **Mistral** definiremos la plantilla de la siguiente forma.
___

In [2]:
def add_prompt(i, df):
    context = df.iloc[i]['context']
    output = df.iloc[i]['output']
    prompt = '<s>[INST]'+context +'[/INST]'+ output + '</s>'
    return prompt

In [5]:
#Cambiamos el nombre
raw_dataset = df.copy()

# Seleccionamos las variables necesarias
raw_dataset = raw_dataset[['context', 'output']]

#Eliminamos los valores None
raw_dataset.dropna(inplace=True)

# Realizamos un Shuffle de los datos para separar entre train/test/validacion
seed = 123
raw_dataset = raw_dataset.sample(frac=1, random_state=seed).reset_index(drop=True)

# Definimos la muestra para el entrenamiento
sample_size = int(len(raw_dataset) * 0.99)

# Realizar el Split para las diferentes muestras
train_data = raw_dataset.iloc[:sample_size].copy()
test_data = raw_dataset.iloc[sample_size:].copy()
vali_data = raw_dataset.iloc[sample_size:].copy()

# Procesamos los datos de acuerdo al formato de Mistral
train_prompts = [add_prompt(i, train_data) for i in range(len(train_data))]
test_prompts = [add_prompt(i, test_data) for i in range(len(test_data))]
vali_prompts = [add_prompt(i, vali_data) for i in range(len(vali_data))]

# Convertimos en formato dataset para poder ser introducidos en la funcion Train
train_dataset = Dataset.from_dict({"text": train_prompts})
test_dataset = Dataset.from_dict({"text": test_prompts})
valid_dataset = Dataset.from_dict({"text": vali_prompts})


# Cargamos las muestras de Train/Test/Validacion
dataset = DatasetDict({"train": train_dataset, "test": test_dataset, "valid": valid_dataset})

In [6]:
train_prompts[0]

'<s>[INST]Consome de ternera[/INST]1 Retire el exceso de grasa de la carne y cortela muy pequeña o pasela por la picadora. 2 Trocee las zanahorias,el apio y los puerros, pongalos en una cacerola grande con el tomate,la carne y las claras de huevo y mezclelos bien. 3 Caliente el caldo hasta que este tibio y salpimentelo al gusto,añada poco a poco el caldo a la carne y a las verduras sin dejar de remover a fuego moderado. 4 Continue removiendo durante unos 10 minutos, hasta que la mezcla empiece a hervir lentamente. 5 Reduzca el fuego,a continuacion haga un agujero en la capa de la grasa que flota en la superficie del caldo con una cuchara grande.dejelo hervir a fuego lento destapado durante 35 minutos y no remueva. 6 Coloque un paño humedo en la boca de un colador grande cuele el liquido y paselo a una olla limpia ,pruebe el consome y añada mas sal al gusto. 7 Recalientelo y sirvalo decorado con tiras muy finas de zanahoria y puerro.</s>'

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2240
    })
    test: Dataset({
        features: ['text'],
        num_rows: 23
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 23
    })
})

<a id="6"></a> 
# 3. Modelo

___

Dado que estamos ejecutando sobre Google Colab vamos a utilizar:

> **Mistral-7B-Instruct-v0.1:** The Mistral-7B-Instruct-v0.1 LLM está optimizado para la conversación y la respuesta a preguntas; la versión instructiva se deriva del modelo de texto generativo Mistral-7B-v0.1. Se ha perfeccionado utilizando varios conjuntos de datos de conversaciones disponibles públicamente.

> **Mistral-7B-Instruct-v0.2:** The Mistral-7B-Instruct-v0.2 LLM es una versión mejorada, instructiva y ajustada de Mistral-7B-Instruct-v0.1.



En nuestro caso por simplicidad del entorno haremos usos de **Mistral-7B-Instruct-v0.1**

---

Cargamos los Token del modelo.

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1")

tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

<a id="6"></a> 
# 4. Cuantificación del Modelo

___

Para alcanzar nuestro objetivo de ajustar **un modelo en una sola GPU**, será necesario **cuantificarlo. Esto implica convertir los pesos del modelo, que originalmente están en formato float32, a un formato más pequeño, en este caso de 4 bits.**

**Para ello, generamos los parámetros de cuantificación utilizando los valores más eficientes:**
 * Cargamos el modelo en 4 bits, empleando el formato NF4 (NormalFloat de 4 bits), un nuevo tipo de dato optimizado para pesos con distribución normal, y aplicando una doble cuantificación, lo que facilita un ahorro significativo de memoria.

___

In [9]:
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

compute_dtype = getattr(torch, "float16")
use_4bit = True

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

device_map = "auto"
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # loading model in 4-bit
    device_map=device_map, # to use max gpu resources if exist
)

#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

<a id="6"></a> 
# 5.QLoRA

___


**QLoRA** es una técnica que optimiza el uso de memoria durante el ajuste fino de modelos de lenguaje grandes sin sacrificar rendimiento. 

**Utiliza cuantización de 4 bits** para comprimir el modelo previamente entrenado, **congelando sus parámetros originales y añadiendo adaptadores de rango bajo entrenables**. Durante el ajuste fino, **QLoRA propaga los gradientes únicamente a través de estos adaptadores**, manteniendo los parámetros congelados sin cambios. Este enfoque permite actualizar **solo** los adaptadores de rango bajo, logrando así un significativo ahorro de memoria y eficiencia en el ajuste fino, comparable al ajuste fino estándar de 16 bits.


**QLoRA** utiliza un tipo de dato para almacenamiento (**NormalFloat de 4 bits**) y otro para cálculo (**BrainFloat de 16 bits**). Los pesos se descomprimen solo cuando son necesarios, manteniendo bajo el uso de memoria durante el entrenamiento y la inferencia.

**Configuracion Parametros LoRA**
* **r** y **lora_aplha** son los parámetros más importantes en la configuración de LoRA.
    * **r** define el rango de matrices de LoRA:
        * Un valor **r más alto** permite que haya más parámetros entrenables, lo que aumenta la capacidad de expresión del modelo. Sin embargo, esto también implica un mayor costo computacional y un riesgo de sobreajuste.
        * Un valor **r más bajo** significa parámetros menos entrenables y puede reducir el sobreajuste a costa de la expresividad.
        
    * **lora_aplha** es un factor de escala para pesos LoRA:
        * Un valor **más alto de lora_aplha** significa que se le dará más énfasis a los pesos de LoRA en el modelo.
        * Un valor **más bajo de lora_aplha** pondrá menos énfasis en los pesos de LoRA, por lo que el modelo dependerá más de sus pesos originales.

Una buena aproximación para definir estos paráemtros es: **lora_aplha:** 2* **r**

**Target_modules** corresponde a los nombres de los módulos que aparecen cuando imprimimos el modelo (q_proj, k_proj, v_proj, etc.)
___


In [10]:
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.1,
        r=32,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
            "lm_head",
        ],
    )

Configuracion parametros LoRA

In [11]:
from transformers import TrainingArguments

run_name = "mistralAI_recetascocina"
training_arguments = TrainingArguments(
    output_dir="./models/"+run_name,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4, # Increase, if still giving OOM error
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    save_steps=500,
    logging_steps=200,
    learning_rate=3e-4,
    fp16=True, # Enable fp16, bf16 only if your gfx card supports it
    evaluation_strategy="steps",
    max_grad_norm=0.3,
    num_train_epochs=1.0,
    weight_decay=0.001,
    warmup_steps=50,
    lr_scheduler_type="linear",
    run_name=run_name,
    report_to="none"
)

<a id="6"></a> 
# 6.Entrenamiento


Como hemos comentado utilizaremos la clase **SFTTrainer(Supervised Fine tuning trainer)** de Hugging Face para llevar a cabo el entrenamiento.

Se ha utilizado esta clase debido a sus caracteristicas:

* **Diseñado para optimizar modelos previamente entrenados utilizando conjuntos de datos más pequeños en tareas de aprendizaje supervisadas.**
* **Proporciona un flujo de trabajo optimizado con menos opciones de configuración, lo que facilita el inicio.**
* **Utiliza técnicas como eficiencia de parámetros (PEFT) y optimizaciones de empaquetado para reducir el consumo de memoria durante el entrenamiento.**
* **Alcanza una precisión similar o superior utilizando conjuntos de datos más pequeños y tiempos de entrenamiento más reducidos en comparación con Trainer.**

In [12]:
import os
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset, # remove you have low VRAM and getting OOM errors
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=2000, # depends on your dataset 4096
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

trainer.train()

Map:   0%|          | 0/2240 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differe

Step,Training Loss,Validation Loss
200,1.719800,1.543985
400,1.545000,1.392581


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=560, training_loss=1.5659747532435826, metrics={'train_runtime': 8788.915, 'train_samples_per_second': 0.255, 'train_steps_per_second': 0.064, 'total_flos': 3.864730548446822e+16, 'train_loss': 1.5659747532435826, 'epoch': 1.0})

<a id="8"></a> 
# 7.Despligue

Finalmente para que podamos utilizar el modelo y realizar las pruebas de validadcion se va a subir el modelo a la cuenta personal de Hugging Face.

Para ello será necesario crear los directorios en la plataforma  para luego apuntar sobre este.

Guardamos el adaptador.

In [25]:
#Modelo preentrando
new_model = "guillergalre/mistralAI_recetascocina"
trainer.save_model(new_model) 
#trainer.model.save_pretrained(new_model)
model.config.use_cache = True
model.eval()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=32, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=32, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplac

Subimos a Hugging Face realizando el commit.

In [29]:
name = 'mistralAI_recetascocina'
trainer.push_to_hub(name)

CommitInfo(commit_url='https://huggingface.co/guillergalre/mistralAI_recetascocina/commit/051d23869e1fd06dc808a504b73b88b937544454', commit_message='mistralAI_recetascocina', commit_description='', oid='051d23869e1fd06dc808a504b73b88b937544454', pr_url=None, pr_revision=None, pr_num=None)